In [19]:
#!pip install transformers
#!pip install torch

In [21]:
import torch
import textwrap
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [22]:
def answer_question(question, answer_text):
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)
    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))
    # Search the input_ids for the first instance of the token.
    sep_index = input_ids.index(tokenizer.sep_token_id)
    # The number of segment A tokens includes the sep token istelf.
    num_seg_a = sep_index + 1
    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a
    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)
    outputs = model(torch.tensor([input_ids]), 
                    token_type_ids=torch.tensor([segment_ids]), 
                    return_dict=True) 

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    # Start with the first token.
    answer = tokens[answer_start]
    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
    print('Answer: "' + answer + '"')

In [23]:
wrapper = textwrap.TextWrapper(width=80) 
bert_abstract = "Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the 'golden anniversary' with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as 'Super Bowl L'), so that the logo could prominently feature the Arabic numerals 50."
question = "Who is champion of The American Football?"
answer_question(question, bert_abstract)

Super Bowl 50 was an American football game to determine the champion of the
National Football League (NFL) for the 2015 season. The American Football
Conference (AFC) champion Denver Broncos defeated the National Football
Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl
title. The game was played on February 7, 2016, at Levi's Stadium in the San
Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl,
the league emphasized the 'golden anniversary' with various gold-themed
initiatives, as well as temporarily suspending the tradition of naming each
Super Bowl game with Roman numerals (under which the game would have been known
as 'Super Bowl L'), so that the logo could prominently feature the Arabic
numerals 50.
Query has 168 tokens.

Answer: "denver broncos"
